# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [1]:
import sqlite3    # Python interface for SQLite databases

In [2]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)


From this point we will use __%%sql__ magic

In [4]:
%load_ext sql
%sql sqlite:///wine.db

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [18]:
%%sql SELECT *
FROM Master1
LIMIT 10;

 * sqlite:///wine.db
Done.


[(None, None, None, None, None, 3, 'Six', 'Paul', 'Alsace'),
 (None, None, None, None, None, 6, 'Marmagne', 'Bernard', 'Bourgogne'),
 (None, None, None, None, None, 8, "Lioger d'Harduy", 'Gabriel', 'Bourgogne'),
 (None, None, None, None, None, 16, 'Barbin', 'Bernard', 'Bourgogne'),
 (None, None, None, None, None, 17, 'Faiveley', 'Guy', 'Bourgogne'),
 (None, None, None, None, None, 18, 'Tramier', 'Jean', 'Bourgogne'),
 (None, None, None, None, None, 19, 'Dupaquier', 'Roger', 'Bourgogne'),
 (None, None, None, None, None, 20, 'Lamy', 'Jean', 'Bourgogne'),
 (None, None, None, None, None, 21, 'Cornu', 'Edmond', 'Bourgogne'),
 (None, None, None, None, None, 26, 'Violot', 'Gilbert', 'Bourgogne')]

* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [15]:
%%sql SELECT *
FROM Master1
WHERE NV = 45;

 * sqlite:///wine.db
Done.


[(45, 'Chiroubles', None, 1983, 90, 2, 'Boxler', 'Albert', 'Alsace'),
 (45, 'Chiroubles', None, 1983, 912, 67, 'Descombes', 'Jean Ernest', 'Beaujolais'),
 (45, 'Chiroubles', None, 1983, 98, 71, 'Chalandard', 'Danile', 'Jura'),
 (45, 'Chiroubles', None, 1983, 540, 78, 'Michlel', 'Pierre Emile', 'Jura'),
 (45, 'Chiroubles', None, 1983, 450, 86, 'Dumazet', 'Marc', 'Rhone')]

---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [19]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

 * sqlite:///wine.db
Done.
Done.


[]

In [20]:
print("\nContent of the database")
printSchema(conn)


Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
dummy:
  0: DEGRE(NUM)


In [21]:
%%sql
SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.


[(None,),
 (11.5,),
 (11.3,),
 (12.1,),
 (10.9,),
 (11.7,),
 (11.2,),
 (12.3,),
 (11.9,),
 (11.8,),
 (10.7,),
 (11.4,),
 (10.5,),
 (11,),
 (11.6,),
 (12,),
 (10,),
 (12.5,),
 (10.8,),
 (14,),
 (11.1,),
 (13,)]

Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [22]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.
Done.
Done.


[(11.5,),
 (11.3,),
 (12.1,),
 (10.9,),
 (11.7,),
 (11.2,),
 (12.3,),
 (11.9,),
 (11.8,),
 (10.7,),
 (11.4,),
 (10.5,),
 (11,),
 (11.6,),
 (12,),
 (10,),
 (12.5,),
 (10.8,),
 (14,),
 (11.1,),
 (13,)]

Notice that we use `NULL` given that `None` is only used for display.

In [23]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:///wine.db
Done.


[]

#### Exercise 1.3

Create the new tables from Master1:

In [46]:
%%sql DROP TABLE IF EXISTS Wine;

-- Create Wine table
CREATE TABLE Wine AS 
SELECT DISTINCT NV,CRU,DEGRE,MILL

FROM MASTER1
WHERE NV IS NOT NULL;

SELECT *
FROM Wine;

 * sqlite:///wine.db
Done.
Done.
Done.


[(1, 'Mercurey', 11.5, 1980),
 (2, 'Julienas', 11.3, 1974),
 (3, 'Savigny les Beaunes', 12.1, 1978),
 (4, 'Mercurey', 10.9, 1980),
 (5, 'Pommard', 11.7, 1976),
 (6, 'Mercurey', 11.2, 1981),
 (7, 'Grands Echezeaux', 11.7, 1968),
 (8, 'Cotes de Beaune Villages', 12.3, 1975),
 (9, 'Chapelle Chambertin', 11.9, 1973),
 (10, 'Beaujolais Villages', 11.8, 1979),
 (11, 'Sylvaner', 11.2, 1982),
 (12, 'Gentil', 10.7, 1975),
 (13, 'Cotes du Rhone', 12.3, 1968),
 (14, 'Chateau Chalon', 10.9, 1978),
 (15, 'Cote de Brouilly', 11.9, 1973),
 (16, 'Meursault', 12.1, 1980),
 (17, 'Meursault', 11.4, 1975),
 (18, 'Chambolle Musigny', 12.3, 1968),
 (19, 'Clos du roi', 11.2, 1972),
 (20, 'Cote de Brouilly', 12.1, 1980),
 (21, 'Cotes de Nuits', None, 1978),
 (22, 'Savigny les Beaunes', None, 1974),
 (23, 'Pommard', None, 1972),
 (24, 'Cotes de Beaune Villages', 12.1, 1971),
 (25, 'Chapelle Chambertin', 10.5, 1968),
 (26, 'Chateau Corton Grancey', None, 1980),
 (27, 'Puligny Montrachet', 11, 1974),
 (28, 'Volnay', 11, 1980),
 (29, 'Corton', 11.6, 1972),
 (30, 'Bourgogne Passe Tout Grains', 12, 1979),
 (31, 'Monthelie', 11.4, 1968),
 (32, 'Pouilly Vinzelles', 12.3, 1968),
 (33, 'Santenay', 12, 1972),
 (34, 'Montagny', 11.2, 1978),
 (35, 'Auxey Duresses', 11.3, 1914),
 (36, 'Chorey les Beaunes', 11, 1955),
 (37, 'Savigny les Beaunes', 10, 1978),
 (38, 'Savigny les Beaunes', 11, 1977),
 (39, 'Savigny les Beaunes', None, 1973),
 (40, 'Richebourg', 12, 1943),
 (41, 'Clairette de Die', 12, 1976),
 (42, 'Beaujolais Primeur', 12, 1983),
 (43, 'Fleurie', 11.4, 1980),
 (44, 'Moulin a vent', None, 1983),
 (45, 'Chiroubles', None, 1983),
 (46, 'Chiroubles', 11, 1983),
 (47, 'Chiroubles', 11, 1978),
 (48, 'Cote de Brouilly', 12, 1899),
 (49, 'Morgon', 12, 1984),
 (50, 'Chenas', 11.5, 1984),
 (51, 'Chenas', 10, 1975),
 (52, 'Brouilly', None, 1975),
 (53, 'Beaujolais Villages', None, 1975),
 (54, 'Beaujolais Villages', 10.5, 1976),
 (55, 'Beaujolais Villages', 12.1, 1978),
 (56, 'Saint Amour', None, 1974),
 (57, 'Crozes Hermitage', 12.1, 1974),
 (58, 'Crozes Hermitage', 12.5, 1975),
 (59, 'Crozes - Hermitage', 12.3, 1972),
 (60, 'Crozes-Hermitage', 11, 1983),
 (61, 'Hermitage', 12, 1979),
 (62, 'Gigondas', 11, 1978),
 (63, 'Coteaux du Tricastin', 10, 1978),
 (64, 'Coteaux du Tricastin', 11, 1974),
 (65, 'Rochgrande', 10.8, 1968),
 (66, 'Tavel', 11, 1968),
 (67, 'Gewurztraminer', 12.5, 1968),
 (68, 'Riesling', 14, 1962),
 (69, "Tokay d'Alsace", 11, 1982),
 (70, 'Sylvaner', 11, 1982),
 (71, 'Palette', 11, 1981),
 (72, 'Cassis', 10, 1978),
 (73, 'Bellet', 11, 1976),
 (74, 'Arbois', 12, 1980),
 (75, 'Arbois', 11, 1976),
 (76, 'Chateau Chalon', 12, 1976),
 (77, 'Etoile', 11, 1979),
 (78, 'Etoile', 12, 1980),
 (79, 'Seyssel', 11, 1980),
 (80, 'Seyssel', 11, 1980),
 (81, 'Cornas', 11.1, 1980),
 (82, 'Cornas', 11, 1980),
 (83, 'Saint Amour', 10, 1980),
 (84, 'Rasteau', 10, 1980),
 (85, 'Tavel', 11, 1980),
 (86, 'Tavel', 11, 1980),
 (87, 'Cotes de Provence', None, 1980),
 (88, 'Cotes de Provence', 10, 1976),
 (89, 'Cotes de Provence', 11, 1969),
 (90, 'Palette', 11, 1969),
 (91, 'Cassis', 11, 1979),
 (92, 'Coteaux de Pierevert', None, 1967),
 (93, 'Blanquette de Limoux', None, 1978),
 (94, 'Cotes du Rousillon', None, 1978),
 (95, 'Cotes du Rousillon', 12, 1978),
 (96, 'Saint Chinian', 11, 1976),
 (97, 'Saint Chinian', 11, 1976),
 (98, 'Corbieres', 10, 1978),
 (99, 'Cote Rotie', 13, 1994),
 (100, 'Seyssel', 11, 1974),
 (101, 'Saint Veran', 12.5, 1973),
 (107, 'Sancerre', 14, 1998)]

In [45]:
%%sql DROP TABLE IF EXISTS Producer;

-- Create Producer table
CREATE TABLE Producer AS 
SELECT DISTINCT NP,NOM,PRENOM,REGION

FROM MASTER1
WHERE NP IS NOT NULL;

SELECT *
FROM Producer;

 * sqlite:///wine.db
Done.
Done.
Done.


[(3, 'Six', 'Paul', 'Alsace'),
 (6, 'Marmagne', 'Bernard', 'Bourgogne'),
 (8, "Lioger d'Harduy", 'Gabriel', 'Bourgogne'),
 (16, 'Barbin', 'Bernard', 'Bourgogne'),
 (17, 'Faiveley', 'Guy', 'Bourgogne'),
 (18, 'Tramier', 'Jean', 'Bourgogne'),
 (19, 'Dupaquier', 'Roger', 'Bourgogne'),
 (20, 'Lamy', 'Jean', 'Bourgogne'),
 (21, 'Cornu', 'Edmond', 'Bourgogne'),
 (26, 'Violot', 'Gilbert', 'Bourgogne'),
 (28, 'Monnier', 'Rene', 'Bourgogne'),
 (29, 'de Suremain', 'Robert', 'Bourgogne'),
 (31, 'Prudhon', 'Henri', 'Bourgogne'),
 (32, 'Mestre', 'Philippe', 'Bourgogne'),
 (33, 'Prieur', 'Guy', 'Bourgogne'),
 (34, 'Monnier', 'Rene', 'Bourgogne'),
 (37, 'Delagrange Bachelet', None, 'Bourgogne'),
 (39, 'de Suremin', 'Hugues', 'Bourgogne'),
 (41, 'Juillot', 'Michel', 'Bourgogne'),
 (44, 'Feyveley', None, 'Bourgogne'),
 (47, 'Thenard', None, 'Bourgogne'),
 (48, 'Flamand', None, 'Bourgogne'),
 (49, 'Guillot', 'Pierre', 'Bourgogne'),
 (51, 'Simonet', 'Leon', 'Bourgogne'),
 (53, 'Cooperative', None, 'Bourgogne'),
 (57, 'Desmules', 'Pierre', 'Beaujolais'),
 (58, 'Joubert', 'Claude', 'Beaujolais'),
 (61, 'Champier', 'Charles', 'Beaujolais'),
 (62, 'Champagnon', 'Louis', 'Beaujolais'),
 (66, 'Cooperative', None, 'Beaujolais'),
 (77, 'Tissot', 'Andre', 'Jura'),
 (83, 'Goy', None, 'Jura'),
 (91, 'Chave', 'Gerard', 'Rhone'),
 (92, 'De Boissieu', None, 'Rhone'),
 (94, 'Grippat', 'Jean Louis', 'Rhone'),
 (96, 'Boisson', 'Jean Pierre', 'Rhone'),
 (97, 'Quiot', 'Pierre', 'Rhone'),
 (100, 'Bernard', 'Georges', 'Rhone'),
 (101, 'Verdan', 'Antoine', 'Rhone'),
 (102, 'Cooperative', None, 'Rhone'),
 (103, 'Coulouvrat', 'Michel', 'Rhone'),
 (104, 'Rabasse Chavarin', 'Abel', 'Rhone'),
 (105, 'Roux', 'Charles', 'Rhone'),
 (106, 'Alary', None, 'Rhone'),
 (108, 'Archimbaud', 'Jean', 'Rhone'),
 (109, 'Rey', 'Marcel', 'Rhone'),
 (110, 'Cooperative', None, 'Rhone'),
 (111, 'Zafiropulo', 'Costa', 'Provence'),
 (112, 'Prebost', 'Gaston', 'Provence'),
 (113, 'Gomez', 'Rene', 'Provence'),
 (114, 'Brunet', 'Georges', 'Provence'),
 (115, 'Bordonado', 'Raymond', 'Provence'),
 (116, 'Imbert', 'Christian', 'Corse'),
 (117, 'Cooperative', None, 'Languedoc'),
 (118, 'Cooperative', None, 'Languedoc'),
 (119, 'Cooperative', None, 'Pyrenees'),
 (121, 'Cooperative', None, 'Pyrenees'),
 (122, 'Dupuy', 'Charles', 'Pyrenees'),
 (123, 'Laurent', 'Pierre', 'Pyrenees'),
 (124, 'Daurat Fort', 'Robert', 'Pyrenees'),
 (125, 'Baldes', 'Jean', 'Sud'),
 (126, 'Jouffreau', 'Jean', 'Sud'),
 (127, 'Brumont', 'Alban', 'Sud'),
 (129, 'Barrere', 'Alfred', 'Sud'),
 (1, 'Bohn', 'Rene', 'Alsace'),
 (73, 'Bourguignon', 'Emile', 'Jura'),
 (5, 'Joudeat', 'Lucien', 'Bourgogne'),
 (10, 'Fournier', 'Jean Claude', 'Bourgogne'),
 (30, 'Desbois', 'Maurice', 'Bourgogne'),
 (42, 'Juillot', 'Maurice', 'Bourgogne'),
 (98, 'Berard', None, 'Rhone'),
 (90, 'Roure', 'Raymond', 'Rhone'),
 (7, 'Lasnier', 'Pierre', 'Bourgogne'),
 (43, 'Menand', 'Jean Claude', 'Bourgogne'),
 (89, 'Michelas', 'Robert', 'Rhone'),
 (13, 'Grivot', 'Jean', 'Bourgogne'),
 (86, 'Dumazet', 'Marc', 'Rhone'),
 (80, 'Devaux', 'Jean Paul', 'Jura'),
 (38, 'de Launay', 'Paul', 'Bourgogne'),
 (95, 'Verset', 'Louis', 'Rhone'),
 (40, 'Genelot', 'Maurice', 'Bourgogne'),
 (50, 'Chagny', 'Georges', 'Bourgogne'),
 (36, 'Sauzet', 'Etienne', 'Bourgogne'),
 (99, 'Cooperative', None, 'Rhone'),
 (22, 'Dubreuil Fontaine', 'Pierre', 'Bourgogne'),
 (9, 'Moniot', 'Gilbert', 'Bourgogne'),
 (72, 'Reverchon', 'Joseph', 'Jura'),
 (60, 'Vernus', 'Georges', 'Beaujolais'),
 (4, 'Stentz', 'Fernand', 'Alsace'),
 (23, 'Laleure', None, 'Bourgogne'),
 (93, 'Marsanne', 'Jean', 'Rhone'),
 (54, 'Cooperative', None, 'Bourgogne'),
 (52, 'Lapierre', 'Jean', 'Bourgogne'),
 (76, 'Maire', 'Henri', 'Jura'),
 (82, 'Mollex', 'Georges', 'Savoie'),
 (85, 'Champet', 'Emile', 'Rhone'),
 (2, 'Boxler', 'Albert', 'Alsace'),
 (67, 'Descombes', 'Jean Ernest', 'Beaujolais'),
 (71, 'Chalandard', 'Danile', 'Jura'),
 (78, 'Michlel', 'Pierre Emile', 'Jura'),
 (64, 'Marius Bel Air', 'Morin', 'Beaujolais'),
 (1

In [73]:
%%sql DROP TABLE IF EXISTS Production;

-- Create Production table
CREATE TABLE Production AS 
SELECT DISTINCT NV,NP,QTE

FROM MASTER1
WHERE NV AND NP IS NOT NULL;

SELECT *
FROM Production;

 * sqlite:///wine.db
Done.
Done.
Done.


[(1, 1, 300),
 (1, 73, 1),
 (2, 5, 100),
 (3, 1, 400),
 (4, 10, 35),
 (5, 30, 46),
 (6, 42, 300),
 (7, 98, 60),
 (8, 90, 12),
 (10, 98, 100),
 (11, 5, 200),
 (12, 7, 31),
 (12, 43, 200),
 (12, 89, 310),
 (13, 13, 13),
 (14, 86, 180),
 (15, 80, 290),
 (16, 38, 100),
 (17, 95, 50),
 (18, 40, 120),
 (19, 7, 986),
 (20, 50, 100),
 (21, 5, 150),
 (22, 86, 50),
 (23, 86, 15),
 (24, 36, 859),
 (25, 95, 40),
 (26, 43, 70),
 (27, 99, 60),
 (28, 50, 10),
 (29, 50, 50),
 (30, 22, 350),
 (31, 50, 320),
 (32, 9, 500),
 (32, 72, 890),
 (33, 60, 70),
 (34, 4, None),
 (35, 10, 900),
 (35, 23, 350),
 (36, 93, 50),
 (37, 54, 830),
 (38, 52, 125),
 (39, 52, 80),
 (41, 76, 25),
 (42, 82, 80),
 (43, 13, 500),
 (43, 72, 900),
 (43, 85, 983),
 (44, 93, 40),
 (45, 2, 90),
 (45, 67, 912),
 (45, 71, 98),
 (45, 78, 540),
 (45, 86, 450),
 (46, 64, 760),
 (47, 52, 75),
 (48, 93, 70),
 (49, 2, 150),
 (50, 2, 100),
 (51, 15, 36),
 (52, 40, 500),
 (53, 40, 800),
 (54, 1, 500),
 (54, 2, 900),
 (54, 14, 500),
 (55, 14, 450),
 (56, 24, 790),
 (57, 40, 940),
 (58, 45, 70),
 (59, 45, 160),
 (60, 50, 300),
 (60, 60, 60),
 (61, 90, 120),
 (62, 60, 900),
 (64, 72, 870),
 (64, 85, 980),
 (65, 12, 10),
 (65, 25, 560),
 (67, 5, 125),
 (67, 10, 890),
 (67, 82, 760),
 (68, 12, None),
 (69, 9, 96),
 (69, 12, 320),
 (69, 69, 690),
 (70, 38, 120),
 (71, 38, 132),
 (72, 52, 34),
 (73, 38, 45),
 (74, 70, 650),
 (74, 72, 850),
 (74, 84, 675),
 (75, 38, 69),
 (76, 46, 100),
 (76, 85, 900),
 (77, 85, 100),
 (78, 4, 300),
 (78, 27, 360),
 (78, 30, 900),
 (78, 68, 980),
 (78, 80, 970),
 (79, 74, 100),
 (79, 79, 79),
 (79, 81, 800),
 (80, 90, 79),
 (81, 90, 24),
 (82, 43, 100),
 (82, 80, 800),
 (83, 95, 423),
 (84, 95, 146),
 (85, 55, 46),
 (86, 93, 500),
 (87, 38, 630),
 (87, 45, 680),
 (87, 72, 900),
 (88, 55, 129),
 (89, 15, 150),
 (89, 42, 980),
 (89, 55, 723),
 (89, 89, 100),
 (90, 56, 900),
 (90, 95, 40),
 (91, 79, 24),
 (92, 11, 900),
 (93, 79, 110),
 (94, 11, 500),
 (94, 27, 180),
 (95, 95, 900),
 (96, 59, 370),
 (97, 70, 101),
 (98, 24, 986),
 (98, 35, 400),
 (98, 75, 150),
 (98, 76, 890),
 (98, 88, 340),
 (99, 5, 100),
 (99, 99, 199),
 (100, 25, 100),
 (101, 70, 213),
 (107, 107, 7000)]

#### Exercise 1.4

Create the new tables from Master2:

In [52]:
%%sql DROP TABLE IF EXISTS Client;

-- Create Client table
CREATE TABLE Client AS 
SELECT DISTINCT NB,NOM,PRENOM,TYPE

FROM MASTER2
WHERE NB IS NOT NULL;

SELECT *
FROM Client;

 * sqlite:///wine.db
Done.
Done.
Done.


[(11, 'Breton', 'Andre', 'petit'),
 (13, 'Barthes', 'Roland', 'moyen'),
 (16, 'Balzac', 'Honore de', 'moyen'),
 (18, 'Celine', 'Louis Ferdinand', 'gros'),
 (20, 'Chateaubriand', 'Francois-Rene de', 'moyen'),
 (21, 'Corbiere', 'Tristan', 'petit'),
 (23, 'Corneille', 'Pierre', 'petit'),
 (25, 'Char', 'Rene', 'petit'),
 (27, 'Dumas', 'Alexandre', 'gros'),
 (29, 'Fournier', 'Alain', 'petit'),
 (32, 'Eluard', 'Paul', 'moyen'),
 (34, 'Mauriac', 'Francois', 'petit'),
 (35, 'Fromentin', 'Eugene', 'gros'),
 (36, 'Maurois', 'Andre', 'petit'),
 (37, 'Freud', 'Sigmund', 'petit'),
 (39, 'Montesquieu', None, 'gros'),
 (42, 'Goethe', 'Johann Wolfgang von', 'moyen'),
 (43, 'Musset', 'Alfred de', 'gros'),
 (46, 'Hugo', 'Victor', 'moyen'),
 (47, 'Gary', 'Romain', 'petit'),
 (51, 'Lafargue', 'Paul', 'petit'),
 (53, 'Lamartine', 'Alphonse de', 'gros'),
 (54, 'Aurele', 'Marc', 'petit'),
 (55, 'Michelet', 'Jules', 'gros'),
 (56, 'Nerval', 'Gerard de', 'petit'),
 (57, 'Marx', 'Karl', 'gros'),
 (59, 'Nietzsche', 'Friedrich', 'petit'),
 (63, 'Maupassant', 'Guy de', 'petit'),
 (64, 'Platon', None, 'gros'),
 (65, 'Pascal', 'Blaise', 'petit'),
 (66, 'Perec', 'Georges', 'gros'),
 (67, 'Nin', 'Anais', 'petit'),
 (68, 'Prevert', 'Jacques', 'gros'),
 (69, 'Queneau', 'Raymond', 'petit'),
 (70, 'Paulhan', 'Jean', 'gros'),
 (71, 'Radiguet', 'Raymond', 'petit'),
 (72, 'Rabelais', 'Francois', 'gros'),
 (73, 'Poe', 'Allan Edgard', 'moyen'),
 (74, 'Rimbaud', 'Arthur', 'petit'),
 (76, 'Racine', 'Jean', 'gros'),
 (77, 'Rousseau', 'Jean-Jacques', 'petit'),
 (78, 'Sandier', 'Jules', 'moyen'),
 (79, 'Sade', 'Donatien Alphonse Fran¿¿s de', 'gros'),
 (81, 'Sartre', 'Jean-Paul', 'gros'),
 (83, 'Sophocle', None, 'gros'),
 (84, 'Sue', 'Eugene', 'moyen'),
 (85, 'Segalen', 'Victor', 'petit'),
 (86, 'Saint Exupery', 'Antoine de', 'gros'),
 (87, 'Segur', 'Comtesse de', 'gros'),
 (88, 'Shakespeare', 'William', 'petit'),
 (89, 'Stendhal', None, 'petit'),
 (90, 'Vian', 'Boris', 'petit'),
 (91, 'Verlaine', 'Paul', 'gros'),
 (93, 'Virgile', None, 'gros'),
 (95, 'Zorn', 'Fritz', 'moyen'),
 (98, 'Maturin', 'Robert', 'gros'),
 (99, 'Cesar', 'Jules', 'gros'),
 (2, 'Artaud', 'Antonin', 'moyen'),
 (44, 'Gide', 'Andre', 'petit'),
 (45, 'Giono', 'Jean', 'petit'),
 (48, 'Hardy', 'Thomas', 'moyen'),
 (7, 'Anouilh', 'Jean', 'moyen'),
 (49, 'Kafka', 'Franz', 'petit'),
 (8, 'Aragon', 'Louis', 'gros'),
 (50, 'Lautreamont', None, 'moyen'),
 (52, 'Carroll', 'Lewis', 'petit'),
 (3, 'Aron', 'Raymond', 'gros'),
 (4, 'Apollinaire', 'Guillaume', 'moyen'),
 (58, 'Marivaux', 'Pierre Carlet de Chamblain de', 'gros'),
 (1, 'Aristote', None, 'petit'),
 (9, 'Ajar', 'Emile', 'petit'),
 (60, 'Ovide', None, 'gros'),
 (5, 'Audiberti', 'Jacques', 'petit'),
 (10, 'Andersen', 'Yann', 'gros'),
 (61, 'Mallarme', 'Stephane', 'petit'),
 (6, 'Arrabal', 'Fernando', 'gros'),
 (62, 'Proust', 'Marcel', 'moyen'),
 (75, 'Ronsard', 'Pierre', 'moyen'),
 (80, 'Sand', 'Georges', 'petit'),
 (82, 'Roussel', 'Rene', 'petit'),
 (100, 'Woolf', 'Virginia', 'petit'),
 (26, 'Diderot', 'Denis', 'moyen'),
 (28, 'Dostoievsky', 'Fiodor', 'gros'),
 (22, 'Cocteau', 'Jean', 'gros'),
 (24, 'Courrier', 'Paul Louis', 'moyen'),
 (30, 'La Fontaine', 'Jean de', 'petit'),
 (12, 'Bataille', 'Georges', 'moyen'),
 (14, 'Bory', 'Jean Louis', 'gros'),
 (15, 'Baudelaire', 'Charles', 'petit'),
 (17, 'Camus', 'Albert', 'petit'),
 (19, 'Ciceron', None, 'gros'),
 (33, 'Flaubert', 'Gustave', 'moyen'),
 (92, 'Verne', 'Jules', 'petit'),
 (31, 'Dante', None, 'gros'),
 (94, 'Zola', 'Emile', 'moyen'),
 (38, 'Montaigne', 'Michel de', 'petit'),
 (96, 'Wilde', 'Oscar', 'petit'),
 (97, 'Vigny', 'Alfred de', 'petit'),
 (40, 'Moliere', None, 'petit'),
 (41, 'Gauthier', 'Theophile', 'petit')]

In [53]:
%%sql DROP TABLE IF EXISTS Place;

-- Create Place table
CREATE TABLE Place AS 
SELECT DISTINCT LIEU,REGION

FROM MASTER2
WHERE LIEU IS NOT NULL;

SELECT *
FROM Place;


 * sqlite:///wine.db
Done.
Done.
Done.


[('BORDEAUX', ' NOUVELLE-AQUITAINE'),
 ('PARIS', ' ÎLE-DE-FRANCE'),
 ('RENNES', ' BRETAGNE'),
 ('LYON', '  AUVERGNE-RHÔNE-ALPES'),
 ('NICE', " PROVENCE-ALPES-CÔTE D'AZUR"),
 ('MERCUREY', ' BOURGOGNE-FRANCHE-COMTÉ'),
 ('TOULOUSE', ' OCCITANIE'),
 ('SENS', ' BOURGOGNE-FRANCHE-COMTÉ'),
 ('LILLE', ' HAUTS-DE-FRANCE'),
 ('NANCY', ' GRAND EST'),
 ('CANNES', " PROVENCE-ALPES-CÔTE D'AZUR"),
 ('TOULON', " PROVENCE-ALPES-CÔTE D'AZUR"),
 ('CHALON SUR SAONE', ' BOURGOGNE-FRANCHE-COMTÉ'),
 ('ROCQUENCOURT', ' ÎLE-DE-FRANCE'),
 ('PARLY', ' BOURGOGNE-FRANCHE-COMTÉ'),
 ('LE CHESNAY', ' ÎLE-DE-FRANCE'),
 ('CAEN', ' NORMANDIE'),
 ('DIJON', ' BOURGOGNE-FRANCHE-COMTÉ')]

In [58]:
%%sql DROP TABLE IF EXISTS Sale;

-- Create Sale table
CREATE TABLE Sale AS 
SELECT NV, NB, DATES, LIEU, QTE

FROM MASTER2
WHERE NV IS NOT NULL and NB IS NOT NULL;

SELECT *
FROM Sale;

 * sqlite:///wine.db
Done.
Done.
Done.


[(1, 2, '1977-11-02', 'BORDEAUX', 33),
 (1, 44, '2015-10-16', 'PARIS', 1),
 (1, 45, '1983-12-31', 'RENNES', 1),
 (2, 48, '1983-12-25', 'LYON', 2),
 (3, 7, '1978-11-01', 'NICE', 6),
 (3, 49, '1987-12-24', 'MERCUREY', 5),
 (4, 8, '1982-11-05', 'TOULOUSE', 12),
 (4, 44, '2015-10-16', 'PARIS', 6),
 (4, 50, '1981-06-29', 'SENS', 4),
 (5, 2, '1984-11-04', 'MERCUREY', 50),
 (5, 52, '1983-06-29', 'LILLE', 5),
 (6, 3, '1978-01-21', 'NANCY', 36),
 (7, 4, '1987-09-19', 'PARIS', 24),
 (7, 58, '1987-04-28', 'CANNES', 7),
 (8, 1, '1982-09-15', 'PARIS', 78),
 (8, 9, '1979-11-01', 'LILLE', 10),
 (8, 60, '1980-05-12', 'TOULON', 8),
 (9, 5, '1982-03-26', 'LYON', 20),
 (9, 10, '1979-05-12', 'LILLE', 1),
 (9, 61, '1907-04-21', 'TOULON', 9),
 (10, 2, '1982-10-30', 'CHALON SUR SAONE', 500),
 (10, 3, '1977-12-31', 'BORDEAUX', 10),
 (10, 5, '1983-06-06', 'LYON', 10),
 (12, 6, '1982-11-05', 'TOULOUSE', 36),
 (16, 44, '2015-10-16', 'PARIS', 2),
 (20, 44, '2015-10-16', 'PARIS', 3),
 (20, 61, '1978-06-12', 'NICE', 20),
 (21, 62, '1987-04-05', 'CANNES', 21),
 (22, 75, '1985-05-23', 'ROCQUENCOURT', 22),
 (22, 80, '1987-12-24', 'PARLY', 22),
 (23, 82, '1988-03-20', 'PARLY', 23),
 (24, 100, '1986-06-12', 'LE CHESNAY', 14),
 (26, 44, '2015-10-16', 'PARIS', 4),
 (28, 44, '2015-10-16', 'PARIS', 5),
 (31, 26, '1984-12-31', 'LYON', 6),
 (31, 28, '1978-12-25', 'LYON', 100),
 (32, 22, '1983-02-14', 'PARIS', 3),
 (32, 24, '1980-12-31', 'PARIS', 7),
 (35, 30, '1981-12-25', 'LYON', 80),
 (38, 12, '1981-08-06', 'CAEN', 5),
 (38, 14, '1981-05-12', 'CAEN', 2),
 (40, 15, '1982-03-25', 'NICE', 30),
 (40, 17, '1982-04-02', 'DIJON', 25),
 (40, 19, '1980-12-31', 'DIJON', 12),
 (40, 33, '1982-04-01', 'TOULOUSE', 1),
 (40, 92, '1986-08-10', 'LE CHESNAY', 4),
 (41, 31, '1988-04-01', 'TOULOUSE', 2),
 (41, 94, '1985-12-29', 'SENS', 3),
 (42, 94, '1985-07-27', 'SENS', 3),
 (42, 100, '1983-03-10', 'PARIS', 4),
 (43, 44, '2015-10-16', 'PARIS', 7),
 (50, 38, '1988-02-23', 'PARIS', 1),
 (53, 12, '1983-06-06', 'ROCQUENCOURT', 10),
 (54, 8, '1983-05-05', 'ROCQUENCOURT', 120),
 (55, 9, '1983-05-05', 'ROCQUENCOURT', 130),
 (56, 5, '1983-05-05', 'ROCQUENCOURT', 80),
 (62, 96, '1980-12-21', 'PARIS', 5),
 (63, 3, '1982-02-02', 'PARIS', 12),
 (64, 97, '1983-12-31', 'PARIS', 1),
 (74, 44, '2015-10-16', 'PARIS', 8),
 (78, 44, '2015-10-16', 'PARIS', 9),
 (79, 44, '2015-10-16', 'PARIS', 10),
 (80, 44, '2015-10-16', 'PARIS', 11),
 (81, 44, '2015-10-16', 'PARIS', 12),
 (82, 44, '2015-10-16', 'PARIS', 13),
 (83, 44, '2015-10-16', 'PARIS', 14),
 (84, 44, '2015-10-16', 'PARIS', 15),
 (85, 44, '2015-10-16', 'PARIS', 16),
 (86, 44, '2015-10-16', 'PARIS', 17),
 (87, 44, '2015-10-16', 'PARIS', 18),
 (100, 5, '1983-02-21', 'CAEN', 3),
 (100, 40, '1982-11-05', 'NANCY', 11),
 (100, 41, '1982-11-04', 'CANNES', 10)]

# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [64]:
%%sql SELECT DISTINCT TYPE AS Type_of_buveurs
FROM Client;

 * sqlite:///wine.db
Done.


[('petit',), ('moyen',), ('gros',)]

#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [76]:
%%sql SELECT DISTINCT CRU, REGION
FROM Wine, Producer,Production
Where (CRU='Brouilly' or CRU='Pommard') and Wine.NV=Production.NV and Producer.NP=Production.NP ;

 * sqlite:///wine.db
Done.


[('Pommard', 'Bourgogne'), ('Pommard', 'Rhone'), ('Brouilly', 'Bourgogne')]

#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [77]:
%%sql SELECT DISTINCT REGION
FROM Wine, Producer,Production
Where (CRU='Brouilly'and Wine.NV=Production.NV and Producer.NP=Production.NP)
INTERSECT
SELECT DISTINCT REGION
FROM Wine, Producer,Production
Where (CRU='Pommard' and Wine.NV=Production.NV and Producer.NP=Production.NP) ;

 * sqlite:///wine.db
Done.


[('Bourgogne',)]

#### Exercise 2.4

Get the number of wines bught by CRU and Millésime

In [79]:
%%sql SELECT CRU, MILL, SUM(Sale.QTE) AS Number_of_wines_bought
FROM Wine, Sale
WHERE Sale.NV=Wine.NV
GROUP BY CRU, MILL;

 * sqlite:///wine.db
Done.


[('Arbois', 1980, 8),
 ('Auxey Duresses', 1914, 80),
 ('Beaujolais Primeur', 1983, 7),
 ('Beaujolais Villages', 1975, 10),
 ('Beaujolais Villages', 1976, 120),
 ('Beaujolais Villages', 1978, 130),
 ('Beaujolais Villages', 1979, 520),
 ('Chapelle Chambertin', 1973, 30),
 ('Chateau Corton Grancey', 1980, 4),
 ('Chenas', 1984, 1),
 ('Clairette de Die', 1976, 5),
 ('Cornas', 1980, 25),
 ('Cote de Brouilly', 1980, 23),
 ('Coteaux du Tricastin', 1974, 1),
 ('Coteaux du Tricastin', 1978, 12),
 ('Cotes de Beaune Villages', 1971, 14),
 ('Cotes de Beaune Villages', 1975, 96),
 ('Cotes de Nuits', 1978, 21),
 ('Cotes de Provence', 1980, 18),
 ('Etoile', 1980, 9),
 ('Fleurie', 1980, 7),
 ('Gentil', 1975, 36),
 ('Gigondas', 1978, 5),
 ('Grands Echezeaux', 1968, 31),
 ('Julienas', 1974, 2),
 ('Mercurey', 1980, 57),
 ('Mercurey', 1981, 36),
 ('Meursault', 1980, 2),
 ('Monthelie', 1968, 106),
 ('Pommard', 1972, 23),
 ('Pommard', 1976, 55),
 ('Pouilly Vinzelles', 1968, 10),
 ('Rasteau', 1980, 15),
 ('Richebourg', 1943, 72),
 ('Saint Amour', 1974, 80),
 ('Saint Amour', 1980, 14),
 ('Savigny les Beaunes', 1974, 44),
 ('Savigny les Beaunes', 1977, 7),
 ('Savigny les Beaunes', 1978, 11),
 ('Seyssel', 1974, 24),
 ('Seyssel', 1980, 21),
 ('Tavel', 1980, 33),
 ('Volnay', 1980, 5)]

#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [100]:
%%sql SELECT NV, Number_of_producers
FROM(SELECT NV, COUNT(NP) AS Number_of_producers
                            FROM Production
                            GROUP BY NV)
WHERE Number_of_producers >3;



 * sqlite:///wine.db
Done.


[(45, 5), (78, 5), (89, 4), (98, 5)]

#### Exercise 2.6

Which producers have not produced any wine?

In [104]:
%%sql SELECT NP,NOM,PRENOM
FROM Producer
WHERE NP IN (SELECT NP
          FROM Producer
          EXCEPT
          SELECT NP
          FROM Production)


 * sqlite:///wine.db
Done.


[(3, 'Six', 'Paul'),
 (6, 'Marmagne', 'Bernard'),
 (8, "Lioger d'Harduy", 'Gabriel'),
 (16, 'Barbin', 'Bernard'),
 (17, 'Faiveley', 'Guy'),
 (18, 'Tramier', 'Jean'),
 (19, 'Dupaquier', 'Roger'),
 (20, 'Lamy', 'Jean'),
 (21, 'Cornu', 'Edmond'),
 (26, 'Violot', 'Gilbert'),
 (28, 'Monnier', 'Rene'),
 (29, 'de Suremain', 'Robert'),
 (31, 'Prudhon', 'Henri'),
 (32, 'Mestre', 'Philippe'),
 (33, 'Prieur', 'Guy'),
 (34, 'Monnier', 'Rene'),
 (37, 'Delagrange Bachelet', None),
 (39, 'de Suremin', 'Hugues'),
 (41, 'Juillot', 'Michel'),
 (44, 'Feyveley', None),
 (47, 'Thenard', None),
 (48, 'Flamand', None),
 (49, 'Guillot', 'Pierre'),
 (51, 'Simonet', 'Leon'),
 (53, 'Cooperative', None),
 (57, 'Desmules', 'Pierre'),
 (58, 'Joubert', 'Claude'),
 (61, 'Champier', 'Charles'),
 (62, 'Champagnon', 'Louis'),
 (66, 'Cooperative', None),
 (77, 'Tissot', 'Andre'),
 (83, 'Goy', None),
 (91, 'Chave', 'Gerard'),
 (92, 'De Boissieu', None),
 (94, 'Grippat', 'Jean Louis'),
 (96, 'Boisson', 'Jean Pierre'),
 (97, 'Quiot', 'Pierre'),
 (100, 'Bernard', 'Georges'),
 (101, 'Verdan', 'Antoine'),
 (102, 'Cooperative', None),
 (103, 'Coulouvrat', 'Michel'),
 (104, 'Rabasse Chavarin', 'Abel'),
 (105, 'Roux', 'Charles'),
 (106, 'Alary', None),
 (108, 'Archimbaud', 'Jean'),
 (109, 'Rey', 'Marcel'),
 (110, 'Cooperative', None),
 (111, 'Zafiropulo', 'Costa'),
 (112, 'Prebost', 'Gaston'),
 (113, 'Gomez', 'Rene'),
 (114, 'Brunet', 'Georges'),
 (115, 'Bordonado', 'Raymond'),
 (116, 'Imbert', 'Christian'),
 (117, 'Cooperative', None),
 (118, 'Cooperative', None),
 (119, 'Cooperative', None),
 (121, 'Cooperative', None),
 (122, 'Dupuy', 'Charles'),
 (123, 'Laurent', 'Pierre'),
 (124, 'Daurat Fort', 'Robert'),
 (125, 'Baldes', 'Jean'),
 (126, 'Jouffreau', 'Jean'),
 (127, 'Brumont', 'Alban'),
 (129, 'Barrere', 'Alfred')]

#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [113]:
%%sql SELECT DISTINCT client.NB,NOM, PRENOM
FROM Client, Wine, Sale
WHERE Client.NB=Sale.NB and Wine.NV=Sale.NV and mill=1980;

 * sqlite:///wine.db
Done.


[(2, 'Artaud', 'Antonin'),
 (44, 'Gide', 'Andre'),
 (45, 'Giono', 'Jean'),
 (8, 'Aragon', 'Louis'),
 (50, 'Lautreamont', None),
 (61, 'Mallarme', 'Stephane')]

#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [114]:
%%sql SELECT NB,NOM, PRENOM
FROM Client 
EXCEPT
SELECT DISTINCT client.NB,NOM, PRENOM
FROM Client, Wine, Sale
WHERE Client.NB=Sale.NB and Wine.NV=Sale.NV and mill=1980;

 * sqlite:///wine.db
Done.


[(1, 'Aristote', None),
 (3, 'Aron', 'Raymond'),
 (4, 'Apollinaire', 'Guillaume'),
 (5, 'Audiberti', 'Jacques'),
 (6, 'Arrabal', 'Fernando'),
 (7, 'Anouilh', 'Jean'),
 (9, 'Ajar', 'Emile'),
 (10, 'Andersen', 'Yann'),
 (11, 'Breton', 'Andre'),
 (12, 'Bataille', 'Georges'),
 (13, 'Barthes', 'Roland'),
 (14, 'Bory', 'Jean Louis'),
 (15, 'Baudelaire', 'Charles'),
 (16, 'Balzac', 'Honore de'),
 (17, 'Camus', 'Albert'),
 (18, 'Celine', 'Louis Ferdinand'),
 (19, 'Ciceron', None),
 (20, 'Chateaubriand', 'Francois-Rene de'),
 (21, 'Corbiere', 'Tristan'),
 (22, 'Cocteau', 'Jean'),
 (23, 'Corneille', 'Pierre'),
 (24, 'Courrier', 'Paul Louis'),
 (25, 'Char', 'Rene'),
 (26, 'Diderot', 'Denis'),
 (27, 'Dumas', 'Alexandre'),
 (28, 'Dostoievsky', 'Fiodor'),
 (29, 'Fournier', 'Alain'),
 (30, 'La Fontaine', 'Jean de'),
 (31, 'Dante', None),
 (32, 'Eluard', 'Paul'),
 (33, 'Flaubert', 'Gustave'),
 (34, 'Mauriac', 'Francois'),
 (35, 'Fromentin', 'Eugene'),
 (36, 'Maurois', 'Andre'),
 (37, 'Freud', 'Sigmund'),
 (38, 'Montaigne', 'Michel de'),
 (39, 'Montesquieu', None),
 (40, 'Moliere', None),
 (41, 'Gauthier', 'Theophile'),
 (42, 'Goethe', 'Johann Wolfgang von'),
 (43, 'Musset', 'Alfred de'),
 (46, 'Hugo', 'Victor'),
 (47, 'Gary', 'Romain'),
 (48, 'Hardy', 'Thomas'),
 (49, 'Kafka', 'Franz'),
 (51, 'Lafargue', 'Paul'),
 (52, 'Carroll', 'Lewis'),
 (53, 'Lamartine', 'Alphonse de'),
 (54, 'Aurele', 'Marc'),
 (55, 'Michelet', 'Jules'),
 (56, 'Nerval', 'Gerard de'),
 (57, 'Marx', 'Karl'),
 (58, 'Marivaux', 'Pierre Carlet de Chamblain de'),
 (59, 'Nietzsche', 'Friedrich'),
 (60, 'Ovide', None),
 (62, 'Proust', 'Marcel'),
 (63, 'Maupassant', 'Guy de'),
 (64, 'Platon', None),
 (65, 'Pascal', 'Blaise'),
 (66, 'Perec', 'Georges'),
 (67, 'Nin', 'Anais'),
 (68, 'Prevert', 'Jacques'),
 (69, 'Queneau', 'Raymond'),
 (70, 'Paulhan', 'Jean'),
 (71, 'Radiguet', 'Raymond'),
 (72, 'Rabelais', 'Francois'),
 (73, 'Poe', 'Allan Edgard'),
 (74, 'Rimbaud', 'Arthur'),
 (75, 'Ronsard', 'Pierre'),
 (76, 'Racine', 'Jean'),
 (77, 'Rousseau', 'Jean-Jacques'),
 (78, 'Sandier', 'Jules'),
 (79, 'Sade', 'Donatien Alphonse Fran¿¿s de'),
 (80, 'Sand', 'Georges'),
 (81, 'Sartre', 'Jean-Paul'),
 (82, 'Roussel', 'Rene'),
 (83, 'Sophocle', None),
 (84, 'Sue', 'Eugene'),
 (85, 'Segalen', 'Victor'),
 (86, 'Saint Exupery', 'Antoine de'),
 (87, 'Segur', 'Comtesse de'),
 (88, 'Shakespeare', 'William'),
 (89, 'Stendhal', None),
 (90, 'Vian', 'Boris'),
 (91, 'Verlaine', 'Paul'),
 (92, 'Verne', 'Jules'),
 (93, 'Virgile', None),
 (94, 'Zola', 'Emile'),
 (95, 'Zorn', 'Fritz'),
 (96, 'Wilde', 'Oscar'),
 (97, 'Vigny', 'Alfred de'),
 (98, 'Maturin', 'Robert'),
 (99, 'Cesar', 'Jules'),
 (100, 'Woolf', 'Virginia')]

#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [122]:
%%sql SELECT DISTINCT client.NB,NOM, PRENOM
FROM Client, Wine, Sale
WHERE Client.NB=Sale.NB and Wine.NV=Sale.NV and mill=1980
EXCEPT
SELECT DISTINCT client.NB,NOM, PRENOM
FROM Client, Wine, Sale
WHERE Client.NB=Sale.NB and Wine.NV=Sale.NV and mill<>1980;

 * sqlite:///wine.db
Done.


[(44, 'Gide', 'Andre'), (45, 'Giono', 'Jean'), (50, 'Lautreamont', None)]

#### Exercise 2.10

List all wines from 1980

In [118]:
%%sql SELECT NV, CRU, MILL, DEGRE
FROM Wine
WHERE MILL=1980;

 * sqlite:///wine.db
Done.


[(1, 'Mercurey', 1980, 11.5),
 (4, 'Mercurey', 1980, 10.9),
 (16, 'Meursault', 1980, 12.1),
 (20, 'Cote de Brouilly', 1980, 12.1),
 (26, 'Chateau Corton Grancey', 1980, None),
 (28, 'Volnay', 1980, 11),
 (43, 'Fleurie', 1980, 11.4),
 (74, 'Arbois', 1980, 12),
 (78, 'Etoile', 1980, 12),
 (79, 'Seyssel', 1980, 11),
 (80, 'Seyssel', 1980, 11),
 (81, 'Cornas', 1980, 11.1),
 (82, 'Cornas', 1980, 11),
 (83, 'Saint Amour', 1980, 10),
 (84, 'Rasteau', 1980, 10),
 (85, 'Tavel', 1980, 11),
 (86, 'Tavel', 1980, 11),
 (87, 'Cotes de Provence', 1980, None)]

#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [127]:
%%sql SELECT Wine.NV, CRU, MILL, DEGRE
FROM Wine, Sale
WHERE MILL=1980 and NB=2 and Wine.NV=Sale.NV;

 * sqlite:///wine.db
Done.


[(1, 'Mercurey', 1980, 11.5)]

#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [174]:
%%sql SELECT Client.NB, NOM, PRENOM
FROM (SELECT COUNT(NV) AS C,NB
             FROM Sale
             GROUP BY NB
             INTERSECT
             SELECT C, NB
             FROM (SELECT COUNT(Wine.NV) AS C
                   FROM Wine
                   WHERE MILL=1980),Client
            ) AS ASW,Client
WHERE Client.NB = ASW.NB;

 * sqlite:///wine.db
Done.


[(44, 'Gide', 'Andre')]